# Phân tích kỹ thuật cho bài toán về dự đoán mức độ trầm cảm ở người đi làm và người đi học

**Phiên bản**: 1.0.0

**Bộ môn**: Chuẩn bị dữ liệu và trực quan hoá

**Giảng viên**: TS. Nguyễn Tuấn Long

**Thành Viên**:

| STT | Họ và tên | Mã sinh viên | Lớp |
|-----|-----------|--------------|-----|
|  1  |    Nguyễn Thị Vân Anh     |        11230603     |  DSEB 65B   |
|  2  |     Nguyễn Mạnh Cường      |        112305      |  DSEB 65B   |
|  3  |     Trần Thu Hiền      |        11230534      |  DSEB 65B   |
|  4  |     Lý Thành Long      |        11230561      |  DSEB 65B   |
|  5  |     Nguyễn Thanh Mơ      |        11230571      |  DSEB 65B   |





**MỤC LỤC**

- [1. Hiểu bài toán, dữ liệu](#1-hiểu-bài-toán-dữ-liệu)
    - [1.1. Từ điển dữ liệu (Data Dictionary)](#11-từ-điển-dữ-liệu-data-dictionary)
- [2. Mô hình cho dữ liệu thô (Baseline Model)](#2-mô-hình-cho-dữ-liệu-thô-baseline-model)
- [3. Xử lý dữ liệu và Feature Engineering](#3-xử-lý-dữ-liệu-và-feature-engineering)
    - [3.1. Làm sạch dữ liệu (Data Cleaning)](#31-làm-sạch-dữ-liệu-data-cleaning)
    - [3.2. Điền dữ liệu thiếu (Imputation)](#32-điền-dữ-liệu-thiếu-imputation)
    - [3.3. Feature Engineering](#33-feature-engineering)
    - [3.4. Feature Selection](#34-feature-selection)
- [4. Mô hình cho dữ liệu đã xử lý (Final Model)](#4-mô-hình-cho-dữ-liệu-đã-xử-lý-final-model)

Nhập các thư viện cần thiết và tải dữ liệu

## 1. Hiểu bài toán, dữ liệu

### 1.1. Từ điển dữ liệu (Data Dictionary)

Dữ liệu bao gồm các thông tin về nhân khẩu học, công việc/học tập, lối sống và tiền sử bệnh lý. Dưới đây là mô tả chi tiết các biến:

| Tên cột | Mô tả | Loại dữ liệu |
|---|---|---|
| Name | Tên người tham gia khảo sát | Object |
| Gender | Giới tính | Object |
| City | Thành phố hiện tại | Object |
| Working Professional or Student | Nghề nghiệp hiện tại (Đi làm hoặc Đi học) | Object |
| Profession | Nghề nghiệp cụ thể | Object |
| Academic Pressure | Áp lực học tập (thang điểm 1-5) | Float |
| Work Pressure | Áp lực công việc (thang điểm 1-5) | Float |
| CGPA | Điểm trung bình tích lũy | Float |
| Study Satisfaction | Mức độ hài lòng với việc học (thang điểm 1-5) | Float |
| Job Satisfaction | Mức độ hài lòng với công việc (thang điểm 1-5) | Float |
| Sleep Duration | Thời gian ngủ trung bình | Object |
| Dietary Habits | Thói quen ăn uống | Object |
| Degree | Bằng cấp | Object |
| Have you ever had suicidal thoughts ? | Có từng có ý định tự tử không? | Object |
| Work/Study Hours | Số giờ làm việc/học tập mỗi ngày | Float |
| Financial Stress | Áp lực tài chính (thang điểm 1-5) | Float |
| Family History of Mental Illness | Tiền sử bệnh tâm thần trong gia đình | Object |
| Depression | Tình trạng trầm cảm (0: Không, 1: Có) | Int |



## 2. Mô hình cho dữ liệu thô (Baseline Model)

Trước khi đi vào xử lý dữ liệu phức tạp, chúng ta sẽ xây dựng một mô hình baseline đơn giản để làm mốc so sánh. 
Cách tiếp cận:
- Loại bỏ cột không quan trọng (Name).
- Chuyển đổi các cột số bị lẫn ký tự (mixed types) sang dạng số (ép kiểu lỗi thành NaN).
- Điền giá trị thiếu bằng 0.
- Sử dụng RandomForestClassifier.

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# 1. Load Data
df = pd.read_csv("raw/raw_depression_dataset.csv")

# 2. Basic Cleaning for Baseline
df_base = df.drop(columns=['Name'])

# Coerce numeric columns
numeric_cols = ['Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Financial Stress']
for col in numeric_cols:
    df_base[col] = pd.to_numeric(df_base[col], errors='coerce')

# 3. Split by Role
df_student = df_base[df_base["Working Professional or Student"] == "Student"].copy()
df_worker = df_base[df_base["Working Professional or Student"] == "Working Professional"].copy()

# 4. Simple Preprocessing (Fillna 0, Select Numeric)
def get_X_y(data):
    data = data.select_dtypes(include=['number']).fillna(0)
    X = data.drop("Depression", axis=1)
    y = data["Depression"]
    return X, y

X_student, y_student = get_X_y(df_student)
X_worker, y_worker = get_X_y(df_worker)

# 5. Train and Evaluate
def train_eval(X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"--- Baseline Results for {name} ---")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))

train_eval(X_student, y_student, "Student")
train_eval(X_worker, y_worker, "Working Professional")

--- Baseline Results for Student ---
Accuracy: 0.7972
              precision    recall  f1-score   support

           0       0.77      0.73      0.75      2814
           1       0.82      0.85      0.83      3971

    accuracy                           0.80      6785
   macro avg       0.79      0.79      0.79      6785
weighted avg       0.80      0.80      0.80      6785

--- Baseline Results for Working Professional ---
Accuracy: 0.9504
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     25109
           1       0.75      0.60      0.66      2247

    accuracy                           0.95     27356
   macro avg       0.86      0.79      0.82     27356
weighted avg       0.95      0.95      0.95     27356



## 3. Xử lý dữ liệu và Feature Engineering

Để cải thiện hiệu suất mô hình, chúng tôi đã thực hiện quy trình xử lý dữ liệu nâng cao như sau:

### 3.1. Làm sạch dữ liệu (Data Cleaning)
- **Chuẩn hóa chuỗi**: Xử lý các lỗi chính tả và không nhất quán trong các cột `City`, `Degree`, `Profession`.
- **Xử lý Outlier**: Cắt bỏ các giá trị ngoại lai vô lý trong cột `Age` (giới hạn 10-90) và `Work/Study Hours`.
- **Chuyển đổi kiểu dữ liệu**: Đảm bảo các cột phân loại (Categorical) có kiểu dữ liệu chuỗi đồng nhất.

### 3.2. Điền dữ liệu thiếu (Imputation)
Thay vì điền 0, chúng tôi sử dụng **Sequential KNN Imputation**:
1.  **CGPA**: Điền dựa trên `Academic Pressure`, `Study Satisfaction`.
2.  **Likert Scales**: Điền các thang đo tâm lý dựa trên `CGPA` đã điền.
3.  **Age**: Điền tuổi dựa trên tất cả các thông tin trên.

### 3.3. Feature Engineering
Tạo ra các đặc trưng mới thể hiện sự tương tác giữa các biến:
- `Age_Pressure`: Tỷ lệ áp lực so với độ tuổi.
- `Burnout_Load`: Tích của áp lực và số giờ làm việc/học tập.
- `Effort_Reward_Imbalance`: Sự mất cân bằng giữa nỗ lực (Burnout) và phần thưởng (Satisfaction).
- `Suicidal_Family_Interaction`: Tương tác giữa ý định tự tử và tiền sử gia đình.

### 3.4. Feature Selection
Sử dụng phương pháp **Aggregated Feature Selection** để chọn ra 10 đặc trưng quan trọng nhất, kết hợp từ 4 phương pháp:
1.  Pearson Correlation
2.  Mutual Information
3.  LassoCV Coefficients
4.  RandomForest Feature Importance

## 4. Mô hình cho dữ liệu đã xử lý (Final Model)

Sử dụng Pipeline đã xây dựng trong `src.pipeline` để tự động hóa toàn bộ quy trình từ làm sạch, điền khuyết, feature engineering, selection đến modeling.
Chúng tôi sẽ chia dữ liệu thành tập Student và Worker, sau đó chia train/test cho từng tập để đánh giá hiệu quả mô hình trên từng nhóm đối tượng.

In [ ]:
from src.pipeline import DepressionAnalysisSystem
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib
import os

# 1. Initialize System
system = DepressionAnalysisSystem()

# 2. Load Data (Raw)
raw_data = pd.read_csv("raw/raw_depression_dataset.csv")

# 3. Correct Roles (Ensure consistency before splitting)
raw_data = system.role_corrector.transform(raw_data)

# 4. Split by Role
student_data = raw_data[raw_data['Working Professional or Student'] == 'Student'].copy()
worker_data = raw_data[raw_data['Working Professional or Student'] == 'Working Professional'].copy()

# 5. Train/Test Split for each subset
print("Splitting data...")
S_train, S_test = train_test_split(student_data, test_size=0.2, random_state=42, stratify=student_data['Depression'])
W_train, W_test = train_test_split(worker_data, test_size=0.2, random_state=42, stratify=worker_data['Depression'])

print(f"Student Train: {S_train.shape}, Test: {S_test.shape}")
print(f"Worker Train: {W_train.shape}, Test: {W_test.shape}")

# 6. Combine Train sets for fitting
# The system handles splitting internally, so we can feed it the combined training data
full_train = pd.concat([S_train, W_train])

print("\nTraining Final Model on combined training set...")
system.fit(full_train)

# 7. Evaluate separately
print("\n--- Evaluation on Student Test Set ---")
s_preds = system.predict(S_test)
print(f"Accuracy: {accuracy_score(S_test['Depression'], s_preds):.4f}")
print(classification_report(S_test['Depression'], s_preds))

print("\n--- Evaluation on Worker Test Set ---")
w_preds = system.predict(W_test)
print(f"Accuracy: {accuracy_score(W_test['Depression'], w_preds):.4f}")
print(classification_report(W_test['Depression'], w_preds))

# 8. Save Model
if not os.path.exists('model'):
    os.makedirs('model')
joblib.dump(system, 'model/full_depression_model.joblib')
print("Model saved to model/full_depression_model.joblib")

Splitting data...
Student Train: (27143, 20), Test: (6786, 20)
Worker Train: (109416, 20), Test: (27355, 20)

Training Final Model on combined training set...
